# Synthetic: Matched Pair Calibration vs Binary Calibration

This example demonstrates how a binary classifier can be perfectly calibrated and still produce unfair rankings.
This example uses a counfounding query type that creates skewed observations, so the net calibration (without accounting for the query type) between item types is identical, yet the ranker consistently undervalues content from item type 1.

In this example, we set relatively arbitrary sets of multiplicative under- and over-valuing, and then compute distributions necessary to achieve calibration despite the intentional bias.

In [29]:
from typing import Callable, Tuple

import numpy as np
import pandas as pd
import plotly.express as px
import scipy.stats as sps
from numpy.lib.recfunctions import merge_arrays

## Configuration

Alter these variables to change the setup of the synthetic example, almost everything should be reasonable.

In [30]:
MPC_PERC_THRESH = 0.01  # percentile for matching scores for MPC
NUM_BATCHES = 16 * 1024  # how many batches to generate
# any distribution that's indepent of viewer group should work,
# to demonstrate this method, but a deterministic number makes
# vectorising easier, and so this whole notebook assumes this is constant.
NUM_ITEMS = 10  # how many items in each ranking batch (n in appendix)

This example generates ranking batches by selecting both a query type $\in \{u, v\}$, and an item type $\in \{1, 2\}$ for every item in a query. We assign multiplicative biases for the label we get conditioned on our predicted score for every pair of query and item type such that the label of $i_1$ items is always greater that similarly scored $i_2$ authors, meaning the ranking is inherently undervaluing $i_1$ (giving them identical score, but they have a higher label).

The biases are multiplicative label biases in first half of the score range [0, 0.5], the bias for the second half of the range is symmetric relative to true calibration. Since these are biases on the label, a bias greater than one indicated undervaluing, that is, the impression resulted in more value than was predicted.

In [31]:
# multiplicative label bias for scores in [0, 0.5] conditioned on query and item type
# 0 < bqvi2 < bqvi1 < 1 < bqui2 < bqui1 < 2
bqui1 = 1.5
bqui2 = 1.1
bqvi1 = 0.9
bqvi2 = 0.7
# probability of a batch belonging to q_u; its possible range is
# dependent on the values above but in a nonlinear way, if it's
# set too low or high an exception will be raised later telling
# you the feasible range conditioned on the biases
pqu = 0.5

In [32]:
ex_score = np.linspace(0, 1, 41)[:, None]
ex_bias = np.array([bqui1, bqui2, bqvi1, bqvi2])[None]
ex_label = np.where(ex_score < 0.5, ex_score * ex_bias, 1 - (1 - ex_score) * (2 - ex_bias))

In [41]:
px.line(
    merge_arrays([
        np.broadcast_to(ex_score, ex_label.shape).flat,
        ex_label.flat,
        np.broadcast_to(np.array(["qᵤ i₁", "qᵤ i₂", "qᵥ i₁", "qᵥ i₂"])[None], ex_label.shape).flat
    ]),
    x="f0",
    y="f1",
    color="f2",
    title="Ground-truth contextual calibration curves for given biases",
    labels={"f0": "Signal", "f1": "Relevance", "f2": "Query-Item Types"},
    width=600,
    height=400,
)

## Generate Data

First we compute some helpful probabilities conditioned on the configuration that will guarantee calibration in observed data despited the presence of the intentional bias. Note that calibration is based on aggregate data, and it's not truely calibrated when you account for the confounding viewer group, and distribution correlations.

In [37]:
# asset necessary configuration properties for this to work
assert 0 < bqvi2 < bqvi1 < 1 < bqui2 < bqui1 < 2, "biases weren't in proper order and ranges"
# p(q_u|i*) necessary so that E[bias|i*] = 1
pqu_i1 = (1 - bqvi1) / (bqui1 - bqvi1)
pqu_i2 = (1 - bqvi2) / (bqui2 - bqvi2)
# these equations assume equal expected impressions per viewer so the math needs to
# change if that assumption changes

# p(q_u) needs to be inbetween these, so that p(i_1) can achieve it by balancing the conditional probabilities
assert pqu_i1 < pqu < pqu_i2, f"pv1 must be in ({pqu_i1:g}, {pqu_i2:g})"
# compute p(i_1) to achieve p(q_u)
pi1 = (pqu - pqu_i2) / (pqu_i1 - pqu_i2)
# use bayes rule to compute p(i_1|v*)
pi1_qu = pqu_i1 * pi1 / pqu
pi1_qv = (1 - pqu_i1) * pi1 / (1 - pqu)

Now we just sample data according to the probabilities we computed

In [38]:
# gather simulated data
np.random.seed(0)
is_qu = np.random.random((NUM_BATCHES, 1)) < pqu  # if viewer is q_u
pi1_b = np.where(is_qu, pi1_qu, pi1_qv)  # p(i_1) for each batch
score = np.random.random((NUM_BATCHES, NUM_ITEMS))  # score for each item iid
is_i1 = np.random.random(score.shape) < pi1_b  # if item is i_1
bias = np.where(is_i1, np.where(is_qu, bqui1, bqvi1), np.where(is_qu, bqui2, bqvi2))
expected_label = np.where(score < 0.5, score * bias, 1 - (1 - score) * (2 - bias))
# NOTE as long as the actual label has the same expectation, any distribution, including constant could be used here
label = (np.random.random(score.shape) < expected_label).astype("f8")

### Calibration

After sampling the data, we graphically verify that calibration is identical between groups.

However, if we break calibration out by viewer, we can detect the bias.

In [42]:
order = np.argsort(score.flat)
i1_score = score.flat[order[is_i1.flat[order]]]
i2_score = score.flat[order[(~is_i1).flat[order]]]
i1_clabel = label.flat[order[is_i1.flat[order]]].cumsum() / i1_score.size
i2_clabel = label.flat[order[(~is_i1).flat[order]]].cumsum() / i2_score.size
b_score = score.flat[order]
b_clabel = label.flat[order].cumsum() / b_score.size

In [46]:
px.line(
    np.concatenate([
        merge_arrays([i1_score, i1_clabel, np.full(i1_score.size, "i₁", "<U4")]),
        merge_arrays([i2_score, i2_clabel, np.full(i2_score.size, "i₂", "<U4")]),
        merge_arrays([b_score, b_clabel, np.full(b_score.size, "both")]),
    ]),
    x="f0",
    y="f1",
    color="f2",
    title="Item types are calibrated",
    labels={
        "f0": "Predicted Score",
        "f1": "Average Cumulative Relevance",
        "f2": "Item Type",
    },
    width=600,
    height=400,
)

In [47]:
order = np.argsort(score.flat)
is_qu_b = np.broadcast_to(is_qu, score.shape)
qu_score = score.flat[order[is_qu_b.flat[order]]]
qv_score = score.flat[order[(~is_qu_b).flat[order]]]
qu_clabel = label.flat[order[is_qu_b.flat[order]]].cumsum() / qu_score.size
qv_clabel = label.flat[order[(~is_qu_b).flat[order]]].cumsum() / qv_score.size
b_score = score.flat[order]
b_clabel = label.flat[order].cumsum() / b_score.size

In [49]:
px.line(
    np.concatenate([
        merge_arrays([qu_score, qu_clabel, np.full(qu_score.size, "qᵤ", "<U4")]),
        merge_arrays([qv_score, qv_clabel, np.full(qv_score.size, "qᵥ", "<U4")]),
        merge_arrays([b_score, b_clabel, np.full(b_score.size, "both")]),
    ]),
    x="f0",
    y="f1",
    color="f2",
    title="Confounding query types are NOT calibrated",
    labels={
        "f0": "Predicted Score",
        "f1": "Average Cumulative Relevance",
        "f2": "Query Type",
    },
    width=600,
    height=400,
)

### Matched Pair Calibration and NDCG

In [12]:
def compute_theory_mpc(
    label: np.ndarray,
    score: np.ndarray,
    group: np.ndarray,
    *,
    perc_thresh: float = MPC_PERC_THRESH,
) -> Tuple[float, float, float]:
    """theoretic mpc error for a set of data

    Each row of input arrays are rows of observations
    """
    # order the batches so that items from the same batche are adjacent and
    # predictions are descending
    order = np.argsort(-score)
    score = np.take_along_axis(score, order, -1)
    label = np.take_along_axis(label, order, -1)
    group = np.take_along_axis(group, order, -1)

    # compute all pairs pairs
    _, dim = label.shape
    finds, sinds = np.triu_indices(dim, 1)

    # find indices of perc_thresh lowest score differences
    pairs = (~group[:, finds]) & (group[:, sinds])
    score_diffs = score[:, finds] - score[:, sinds]
    thresh = np.quantile(score_diffs[pairs], perc_thresh)
    is_mp = pairs & (score_diffs <= thresh)

    # compute label differences for lowest score indices
    paired_label_diffs = label[:, sinds][is_mp] - label[:, finds][is_mp]

    # compute mpc
    lower, upper = sps.bootstrap(paired_label_diffs[None], np.mean).confidence_interval
    return lower, paired_label_diffs.mean(), upper

In [14]:
def compute_practice_mpc(
    label: np.ndarray,
    score: np.ndarray,
    group: np.ndarray,
    *,
    perc_thresh: float = MPC_PERC_THRESH,
) -> Tuple[float, float, float]:
    """practical mpc error for a set of data

    All inputs are parallel rows of batch data
    """
    order = np.argsort(-score)
    score = np.take_along_axis(score, order, -1)
    label = np.take_along_axis(label, order, -1)
    group = np.take_along_axis(group, order, -1)

    # compute batches
    pairs = group[:, :-1] != group[:, 1:]

    # compute the percentile
    score_diffs = -np.diff(score, 1, 1)
    thresh = np.quantile(score_diffs[pairs], perc_thresh)

    # compute matched pairs, and the corresponding label differences
    is_mp = pairs & (score_diffs <= thresh)
    signed_diffs = np.diff(label, 1, 1) * np.where(group[:, 1:], 1, -1)
    paired_label_diffs = signed_diffs[is_mp]

    # compute mpc
    lower, upper = sps.bootstrap(paired_label_diffs[None], np.mean).confidence_interval
    return lower, paired_label_diffs.mean(), upper

In [16]:
def compute_ndcg(score: np.ndarray, label: np.ndarray) -> Tuple[float, float, float]:
    """computes mean ndcg with confidence bounds

    Each row in inputs is treated as a batch.
    """
    _, dim = label.shape
    order = np.argsort(-score)
    oscore = np.take_along_axis(score, order, -1)
    olabel = np.take_along_axis(label, order, -1)
    slabel = np.sort(label, -1)[..., ::-1]
    disc = 1 / np.log2(np.arange(2, dim + 2))
    # add small eps for 0 labels so that we return 1 ndcg
    eps = np.finfo(label.dtype).eps
    ndcgs = (olabel @ disc + eps) / (slabel @ disc + eps)
    lower, upper = sps.bootstrap(ndcgs[None], np.mean).confidence_interval
    return lower, ndcgs.mean(), upper

In [13]:
lower, mpc, upper = compute_theory_mpc(label, score, is_i1)
print(f"theory mpc: {mpc:.4f} ({lower:.4f} - {upper:.4f})")

In [15]:
lower, mpc, upper = compute_practice_mpc(label, score, is_i1)
print(f"practice mpc: {mpc:.4f} ({lower:.4f} - {upper:.4f})")

In [17]:
lower, ndcg, upper = compute_ndcg(score, label)
print(f"baseline ndcg: {ndcg:.4f} ({lower:.4f} - {upper:.4f})")

In [22]:
boosted_scores = np.where(is_i1, 1 - (1 - score) * 0.9, score)

In [23]:
lower, mpc, upper = compute_theory_mpc(label, boosted_scores, is_i1)
print(f"i1-boosted theory mpc: {mpc:.4f} ({lower:.4f} - {upper:.4f})")

In [25]:
lower, mpc, upper = compute_practice_mpc(label, boosted_scores, is_i1)
print(f"i1-boosted practice mpc: {mpc:.4f} ({lower:.4f} - {upper:.4f})")

In [27]:
lower, ndcg, upper = compute_ndcg(boosted_scores, label)
print(f"i1-boosted ndcg: {ndcg:.4f} ({lower:.4f} - {upper:.4f})")